<H1 align = 'center'>Digit Recognization - CNN by using Keras</H1>
<img src = 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/Plot-of-a-Subset-of-Images-from-the-MNIST-Dataset.png' alt = 'Digit Recognization'></img>


* <strong>If you liked the worked, an upvote will be nice!</strong>
* <strong>Any suggestions and feedback are always welcome</strong>
* <strong>If you have any doubt, let's dicuss in the comments</strong>

# Table of Contents

# Content:

* [Introduction](#Introduction)
* [MNIST Dataset](#MNIST)
* [Importing  Libraries](#library)
* [Loading the Dataset](#load)
* [Visualisation](#visualise)
* [Data Preperation](#prepare)
* **[Model I](#model1)**
    * [Model building](#Bmodel1)
    * [Compiling Model](#compile1)
    * [Training the Model](#train1)
    * [Training Performance](#perform1)
* **[Model II](#model2)**
    * [Model building](#Bmodel2)
    * [Compiling Model](#compile2)
    * [Data Agumentation](#agument2)
    * [Training the Model](#train2)
    * [Training Performance](#perform2)
    * [Predicting the test data](#predict)
* **[Model III](#model3)**
    * [Model building](#Bmodel3)
    * [Compiling Model](#compile3)
    * [Data Agumentation](#agument3)
    * [Training the Model](#train3)
    * [Training Performance](#perform3)
* [Conclusion](#conclude)

<a id="Introduction"></a>
## Introduction

* In this dataset, I will solve the digit recognization problem
* We will solve the problem by using CNN, I highly suggest to read as much as you can and make an intutive understanding of CNN before solving this problem
* No matter where you look, you will find complex neural networks architechture which can be very confusing
* Sometimes you don't need a very complex model a simple model can also do the work. 
* Our aim should be to make a better model not complex model
* I will start with a very simple architechture and further make it complex to get better accuracy and better fitted model
* I will explain each and everything as to why I added the component to the model and how why I took the decision to do so

<a id="MNIST"></a>
## MNIST Dataset
<p>The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.
<br>
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.
</p>

<a id="library"></a>
<H2>Importing Libraries</H2>
<H3>In this notebook the main libraries we will import are</H3>
<ol>
    <li><B>Pandas</B>:For handling the CSV files</li>
    <li><B>Numpy</B>:Numerical calculations</li>
    <li><B>Matplotlib/Seaborn</B>:Visualisation</li>
    <li><B>Keras</B>: Building  the CNN model and prediction</li>
</ol>
<p>I will import the libraries as it is used rather than just importing  in the  start , this will help you understand where the specific module is used in the notebook</p>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<a id = 'load'></a>
## Loading the Dataset

In [ ]:
#Importing  the  dataset using pandas 
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
#Checking the first 5 columns in the train dataset
print(train.head())
print('Shape of the train is {}'.format(train.shape))
print('Shape of the test is {}'.format(test.shape))

* Dataset size 70k samples of handwritten images of which 42k is in train with the label and rest 28k is in test dataset
* The dataset is dataframe with 784 columns which is  square of 28 this implies the size of each image is 28x28 pixels.
* Each image has only 1 color channel, i.e., grayscale image.
* Each pixel has value in the range of [0,255] where 0 represents black, and 255 represents white.
* Each image has labeled from 0-9.

<a id ='visualise'></a>
## Visualisation

<p>First, we will start with visualising the  dependent varaible distribution with the countplot</p>


In [ ]:
# Visualising the label using the  bar plot to get the  idea of the  distribution of the target feature
import seaborn as sns
sns.countplot(train['label'])

* <p>Here the distribution of the dependent variable seems balanced which  is good for  our model</p>
* <p>It's not always the  case when  you  will get balanced dataset</p>
* <p>In case of unbalanced dataset, we will have to use the sampling techniques but that's not the case with the dataset</p>

Seperating the indendent feature from the dependent feature

In [ ]:
#Breaking the training dataset into matrix of independent features and dependent feature vector
X = train.drop('label',axis = 1)
y = train.label

In [ ]:
# Let's check the shape 
print('Shape of the X is {}'.format(X.shape))
print(f'Shape of y is {y.shape}')

In [ ]:
#Using the matplotlib to display the image and the  label of the  image 
import matplotlib.pyplot as plt
plt.imshow(X.values.reshape(-1,28,28)[3],cmap = 'gray')
plt.show()
print(y[3])

Clearly, this image of '4' and it is labelled as 4

<a id = 'prepare'></a>
# Data preparation

In [ ]:
#Reshaping the size of the image to 28 x 28 pixels
X = X.values.reshape(-1,28,28,1)

Train and test images (28px x 28px) has been represented as dataframe as 1D vectors of 784 values. We reshape all data to 28x28x1 3D matrices.
<br>
Keras requires an extra dimension in the end which correspond to channels. MNIST images are gray scaled so it use only one channel. For RGB images, there is 3 channels, we would have reshaped 784px vectors to 28x28x3 3D matrices.

In [ ]:
#Rechecking the shape of the image
print('Shape of the X is {}'.format(X.shape))

One hot encoding is needed as the model expects each of the labels to be in array of 10 elements in which only one of the elements = 1 and rest of the elements = 0<br>
Labels are 10 digits numbers from 0 to 9. It is  encoded to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0]

In [ ]:
#Converting the dependent feature vector into one hot  encoding 
from keras.utils import to_categorical
y = to_categorical(y)

<p>We will let our model learn on the training set and validate it's performance with the validation set</p>
We are spliting the train set in two parts : (20%) of the train dataset as the validation set which the model is evaluated and the rest (80%) is used to train the model.

In [ ]:
#Dividing the dataset into training and validation set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 123)

<a id = 'model1'></a>
# Model I

<a id = 'Bmodel1'></a>
## Building Model

* The model type that we are using is sequential. 
* It is one of the easiest way to build a CNN model in keras. It helps to build the in successive layers  with the add() function.
* The first layer is the input of the shape which is 28 x 28 x 1, in this case
* Model is started with Conv2D layers. This will enable the convulation layers to deal with our input images which are seen as 2D matrices
* The nodes used in the layers are 32 and 64, these numbers are choosen  arbitarily  and can be adjusted according to the size of the dataset
* Kernel size is 3, which is 3x3 filter matrix for extracting the information from the image
* Activation function used here is Relu in between the layers, this activation function is widely used in the neural network and is known for the speed and output in the neural nets. 
* Next there is flatten layer which is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.
* In the end  there is dense layer which  is used for output layer, the activation used in the last layer is softmax, softmax returns the prediction with the maximum probablity of the dependent feature
* Once our layers are added to the model, we need to set up a score function, a loss function and an optimisation algorithm.
* We define the loss function to measure how poorly our model performs on images with known labels. It is the error rate between the oberved labels and the predicted ones. 
* The most important function is the optimizer. This function will iteratively improve parameters (filters kernel values, weights and bias of neurons ...) in order to minimise the loss.
* The metric function "accuracy" is used is to evaluate the performance our model. This metric function is similar to the loss function, except that the results from the metric evaluation are not used when training the model (only for evaluation).
* <b>Optimizer</b> — It controls the learning rate. We will be using ‘adam’ optimizer. It is a very good optimizer as it utilises the perks of both Stochastic gradient and RMSprop optimizers.
* <b>Loss function</b> — We will be using ‘categorical_crossentropy’ loss function. It is the most common choice for classification. A lower score corresponds to better performance.
* <b>Metrics</b> — To make things easier to interpret, we will be using ‘accuracy’ metrix to see the accuracy score on the validation set while training the model.

In [ ]:
#Model building using keras
from keras.models import Sequential
from keras.layers import Dense, Flatten,MaxPooling2D,Dropout
from keras.layers.convolutional  import Conv2D

model = Sequential()

model.add(Conv2D(32,kernel_size = (3,3),activation = 'relu',input_shape = (28,28,1)))
model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu'))
model.add(Flatten())
model.add(Dense(10,activation = 'softmax'))


<a id='compile1'></a>
## Compiling Model

In [ ]:
#Compiling  the  model
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

<a id = 'train1'></a>
## Training the model

* Now will fit the model to the training data and will be validated against the  validated data
* epochs = 20
* 1 epoch -> One iteration/cycle of the dataset throughout the Neural Network

In [ ]:
#Training
history = model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 20)

<a id = 'perform1'></a>
## Training Performance.

* Plotting the accuracy and loss of both training and validation set with each epoch to check the training performance. 
* In the accuracy graphs, there is clearly a difference in the training and validation set. 
* The model is more accurate on the training set. 
* Even though you see the model is not complex it has done a fairly well job
* Also while  evaluating the performace of the  model also check the training and validation loss, both the loss should be as low as possible for better model
* Let's try to max a bit more complex model by doing some add ons.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(acc)) 

# Plotting training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

# Plotting training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.legend()
plt.title('Training and validation loss')

<a id = 'model2'></a>
# Model II

<a id = 'Bmodel2'></a>
## Building the Model

* Now we make a new sequential model for better fit to our data

* The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. 
* Here we are setting 32 filters for the two firsts conv2D layers and 64 filters for the two last ones. 
* Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

* The CNN isolates the features that are useful everywhere from these transformed images (feature maps).

* The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

* Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a propotion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting.

* 'relu' is the rectifier. The rectifier activation function is used to add non linearity to the network.

* The Flatten layer is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.

* In the end i used the features in two fully-connected (Dense) layers which is just artificial an neural networks (ANN) classifier. In the last layer(Dense(10,activation="softmax")) the net outputs distribution of probability of each class.

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

* **Optimizer** — It controls the learning rate. We will be using ‘adam’ optimizer. It is a very good optimizer as it utilises the perks of both Stochastic gradient and RMSprop optimizers.
* **Loss function** — We will be using ‘categorical_crossentropy’ loss function. It is the most common choice for classification. A lower score corresponds to better performance.
* **Metrics** — To make things easier to interpret, we will be using ‘accuracy’ metrix to see the accuracy score on the validation set while training the model.

<a id = 'compile2'></a>
## Compiling the Model

In [ ]:
# Compile the model
#from keras.optimizers import RMSprop
#optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

<a id ='agument2'></a>
## Data Agumentation

* The idea behind data augmentation is that we can artificially enlarge our training dataset (thus reducing overfitting) by its augmentation.
* The idea is to alter the training data with small transformations to reproduce the variations occuring when someone is writing a digit.
<img src = 'https://cdn-images-1.medium.com/max/1000/1*dJNlEc7yf93K4pjRJL55PA.png' alt = 'Image agumentation'>
<h3>Parameters in the  data agumentation</h3>
1. **rotation_range**: randomly rotate images in the range (degrees, 0 to 180) 
2. **zoom_range**: Randomly zoom image
3. **width_shift_range**: randomly shift images horizontally (fraction of total width)
4. **height_shift_range**: randomly shift images vertically (fraction of total height)
5. **horizontal_flip**: randomly flip images (Can't be used in this case as it changes the digit)
6. **vertical_flip**: randomly flip images (Can't be used in this case as it changes the digit

In [ ]:
#Data agumentation to preventing the overfitting
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  


datagen.fit(X_train)

<a id = 'train2'></a>
## Training the Model

In order to make the optimizer converge faster and closest to the global minimum of the loss function, we will use an annealing method of the learning rate (LR).

The LR is the step by which the optimizer walks through the 'loss landscape'. The higher LR, the bigger are the steps and the quicker is the convergence. However the sampling is very poor with an high LR and the optimizer could probably fall into a local minima.

Its better to have a decreasing learning rate during the training to reach efficiently the global minimum of the loss function.

To keep the advantage of the fast computation time with a high LR, i decreased the LR dynamically every X steps (epochs) depending if it is necessary (when accuracy is not improved).

With the ReduceLROnPlateau function from Keras.callbacks, i choose to reduce the LR by half if the accuracy is not improved after 3 epochs.

In [ ]:
# Set a learning rate annealer
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
batch_size = 256
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = 30, validation_data = (X_test,y_test),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

<a id = 'perform2'></a>
##  Training performance
* This model has definetly performed better than the previous model 
* The accuracy achived here  is 99.50%
* Also the training and validaiton loss both are minimised which implies there is no overfitting and underfitting
* Remember we always want just fit model, we don't need extremely perfect model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs   = range(len(acc)) 

# Diffining Figure
fig = plt.figure(figsize=(20,7))

#Subplot 1 (For Accuracy)
fig.add_subplot(121)

plt.plot(epochs, acc,label = 'Accuracy')
plt.plot(epochs, val_acc,label = 'Validation Accuracy')
plt.title("Accuracy Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Accuracy",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

#Subplot 2 (For Loss)
fig.add_subplot(122)
plt.plot(epochs, loss,label = 'Loss')
plt.plot(epochs, val_loss,label = 'Validation Loss')
plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.show()

<a id= 'predict'></a>
## Predicting the test data

In [ ]:
#Reshaping the test dataset
test = test.values.reshape(-1,28,28,1)

In [ ]:
#Predicting the test using our model
predictions = model.predict_classes(test, verbose=1)

In [ ]:
predictions

In [ ]:
#Converting our predictions into dataframe
prediction = pd.DataFrame({"ImageId":list(range(1,len(predictions)+1)),"Label":predictions})

In [ ]:
#Finally exporting to the csv file as the output
prediction.to_csv('kaggle_submission.csv',index=False,header=True)
prediction

<a id = 'model3'></a>
# Model III
let's go one step further and add some more add on in the model and see how it perform

<a id= 'Bmodel3'></a>
## Building  the Model

* Now we make a new sequential model for better fit to our data

* The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. 
* Here we are setting 64 filters for the two firsts conv2D layers and 28 filters for the two last ones. 
* Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

* The CNN isolates the features that are useful everywhere from these transformed images (feature maps).

* The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

* Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a propotion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting.

* Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process and dramatically reducing the number of training epochs required to train deep networks.

* 'relu' is the rectifier. The rectifier activation function is used to add non linearity to the network.

* The Flatten layer is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.

* In the end i used the features in two fully-connected (Dense) layers which is just artificial an neural networks (ANN) classifier. In the last layer(Dense(10,activation="softmax")) the net outputs distribution of probability of each class.

* We define the loss function to measure how poorly our model performs on images with known labels. It is the error rate between the oberved labels and the predicted ones. We use a specific form for categorical classifications (>2 classes) called the "categorical_crossentropy".

* The most important function is the optimizer. This function will iteratively improve parameters (filters kernel values, weights and bias of neurons ...) in order to minimise the loss.

* we have choosen the RMSprop (with default values), it is a very effective optimizer. The RMSProp update adjusts the Adagrad method in a very simple way in an attempt to reduce its aggressive, monotonically decreasing learning rate. We could also have used Stochastic Gradient Descent ('sgd') optimizer, but it is slower than RMSprop.

* The metric function "accuracy" is used is to evaluate the performance our model. This metric function is similar to the loss function, except that the results from the metric evaluation are not used when training the model (only for evaluation).

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten,MaxPooling2D,Dropout,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.layers.convolutional  import Conv2D

model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(10,activation='softmax'))



<a id = 'compile3'></a>
## Compiling the Model

In [ ]:
#Compiling the model
model.compile(RMSprop(lr=0.001,rho=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

<a id = 'agument3'></a>
## Data Agumentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=False,
                                   fill_mode='nearest')
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train,y_train,batch_size=128)


<a id = 'train3'></a>
## Training the Model

In [ ]:
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

earlystop = EarlyStopping(monitor='val_loss',patience=2,verbose=1)
learning_reduce = ReduceLROnPlateau(patience=2,monitor="val_acc",verbose=1,min_lr=0.00001,factor=0.5)
#callbacks = [earlystop,learning_reduce]
callbacks = [learning_reduce]
history = model.fit_generator(train_generator,epochs=30,verbose=1,validation_data=(X_test,y_test),callbacks=callbacks)

<a id = 'perform3'></a>
# Model performance
* Here the accuracy is 99.11% which is not bad, but this way we understand we always don't need a very complex and deep neural network
* The neural network architechture is always based on the intution of the dataset it is always important to learn the dataset throughly before creating a model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs   = range(len(acc)) 

# Diffining Figure
fig = plt.figure(figsize=(20,7))

#Subplot 1 (For Accuracy)
fig.add_subplot(121)

plt.plot(epochs, acc,label = 'Accuracy')
plt.plot(epochs, val_acc,label = 'Validation Accuracy')
plt.title("Accuracy Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Accuracy",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

#Subplot 2 (For Loss)
fig.add_subplot(122)
plt.plot(epochs, loss,label = 'Loss')
plt.plot(epochs, val_loss,label = 'Validation Loss')
plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

plt.show()

<a id = 'conclude'></a>
## Conclusion
* That's all Folks
* I highly encourage you all to try building your own model with the intution you have developed with the notebook
* All the best

# Feedback
* <strong>If you liked the worked, an upvote will be nice!</strong>
* <strong>Any suggestions and feedback are always welcome</strong>
* <strong>If you have any doubt, let's dicuss in the comments</strong>